# Inciso 5: Analice los resultados del modelo de regresión ¿Qué tan bien le fue prediciendo?

 Por lo general esto se vería como que es bueno prediciendo los datos aunque puede haber cierto caso de overfitting. En caso se tengan datos nuevos, estos puede que no vayan a poder ser tan bien predecidos como se debería esperar. Sin embargo, por el momento como predictor de precios en el area que se esta trabajando si sirve como un buen modelo de regresión y nos provee una buena medida para predecir los precios de las casas. 



# inciso 6: Compare los resultados con el modelo de regresión lineal y el arbol de regresión que hizo en las hojas pasadas, ¿Cuál funcionó mejor?

Comparando con el modelo de clasificación  (arbol de regresión) que se tuvo una precisión del 0.72 aproximadamente, este modelo de regresión naive bayes cuenta con una precisión de 0.96. Incluso si se comapra con el modelo de regresión lineal, este modelo de regresión naive excede las expectativas ya que el modelo de regresión lineal solo llego a un 0.65 de efectividad en cuanto a sus predicciones.

# inciso 7: Haga un análisis de la eficiencia del modelo de clasificación utilizando una matriz de confusión. Tenga en cuenta le efectividad, donde el algoritmo se equivocó más, donde se equivocó menos y la importancia que tienen los errores.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder


datos = pd.read_csv("train.csv", encoding="latin1")

In [2]:
# Seleccionar solo las variables numéricas relevantes
numericas = datos.select_dtypes(include='number')
numericas = numericas.drop(["Fireplaces", "GarageYrBlt", "Id", "MSSubClass", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"], axis=1)

In [3]:
# Crear variable respuesta que clasifica las casas en Económicas, Intermedias o Caras
terciles = np.percentile(numericas["SalePrice"], [33.33, 66.67])
limite_economicas = terciles[0]
limite_caras = terciles[1]
numericas["Clase"] = pd.cut(numericas["SalePrice"], bins=[0, limite_economicas, limite_caras, float("inf")], labels=["Económicas", "Intermedias", "Caras"])

# Mostrar clasificación de las casas
print(numericas["Clase"].value_counts())

Intermedias    490
Económicas     487
Caras          483
Name: Clase, dtype: int64


In [4]:
numericas = numericas.dropna()
X = numericas.drop("Clase", axis=1)
y = numericas["Clase"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [7]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

c:\Users\Alien Ware\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [10]:
y_pred = clf.predict(X_test)

tp = confusion_matrix(y_test, y_pred)
tp

array([[79,  0,  4],
       [ 0, 82,  2],
       [ 1,  1, 70]], dtype=int64)

In [16]:
print("Precision económicas: ", (tp[0][0]/tp[0].sum()))
print("Precision intermedias: ", (tp[1][1]/tp[1].sum()))
print("Precision caras: ", (tp[2][2]/tp[2].sum()))
print(" ")
print("Recall económicas: ", (tp[:, 0][0]/tp[:,0].sum()))
print("Recall intermedias: ", (tp[:,1][1]/tp[:,1].sum()))
print("Recall caras: ", (tp[:,2][2]/tp[:,2].sum()))


Precision económicas:  0.9518072289156626
Precision intermedias:  0.9761904761904762
Precision caras:  0.9722222222222222
 
Recall económicas:  0.9875
Recall intermedias:  0.9879518072289156
recall caras:  0.9210526315789473


Como se puede observar en la matriz de confusión se tiene al clasificar todas las casas se tuvo una precisión del 0.96 en promedio. Lo cual nos dice que por lo general hay una buena clasificación de los datos y que además los datos estan balanceados y por ello no hay tantos errores. Sin embargo, cuando se observa el recall de las caras hay una leve disminucioón aunque no es muy significativa ya que aun proporciona un 0.92 de recall. 

Estos valores en donde se equivocaron las predicciones puede deberse a que el precio estaba entre un punto intermedio entre las 3 opciones y por lo tanto por sus distintas características, no pudo predecir correctamente su categoría. 